# Multiple processes (hierarchy)

_P0_ & _P1_ --> || --> _P2_ --> _P3_ --> || --> _P5_
        

In [1]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.ports.ports import InPort, OutPort

As first step we define the _Processes_ with their respective _Ports_ _out_ and _inp_.

In [2]:
# Processes with an OutPort (senders).
class P0(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.out0 = OutPort(shape=shape)

class P1(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.out1 = OutPort(shape=shape)

# Processes that will be part of a bigger process (hiercharchical processes).
class P2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp2 = InPort(shape=shape)
        self.out2 = OutPort(shape=shape)

class P3(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp3 = InPort(shape=shape)
        self.out3 = OutPort(shape=shape)

# Process with an Inport (receiver).
class P5(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp5 = InPort(shape=shape)
        self.out5 = OutPort(shape=shape)

Each process requires a corresponding _ProcessModel_ which implements their _Ports_ and a RunConfig for sending and receiving data.

In the _ProcessModels_ Python code should be exectued on a CPU. The input and output _Port_ should be able to receive/send a vector of integers and print the transferred data.

So the _ProcessModel_ inherits form _AbstractPyProcessModel_ in order to execute Python code and the configured _ComputeResource_ is a CPU. A _LavaPyType_ is used for the _Ports_. The _LavaPyType_ specifies the expected data format for the _Port_. A dense vector of type integer is chosen with the parameters _PyOutPort.VEC_DENSE_ and _int_. The _Ports_ can be used to send and receive data by calling _send_ or _recv_. The sent and received data is afterwards printed out.

In [3]:
import numpy as np
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.resources import CPU
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol

In [4]:
#PyProcModel implementing P0
@implements(proc=P0, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelA0(PyLoihiProcessModel):
    out0: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)

    def run_spk(self):
        data = np.array([1.2, 1])
        print("Sent output data of P0: {}\n".format(data))
        self.out0.send(data)

#PyProcModel implementing P1
@implements(proc=P1, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelA(PyLoihiProcessModel):
    out1: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)

    def run_spk(self):
        data = np.array([1, 2])
        print("Sent output data of P1: {}\n".format(data))
        self.out1.send(data)


#PyProcModel implementing P2
@implements(proc=P2, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelC(PyLoihiProcessModel):

    inp2: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    out2: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)
    def run_spk(self):
        in_data2 = self.inp2.recv()
        print("P2 received: {}\n".format(in_data2))
        out_data2=in_data2*2
        print("Sent output data of P2: {}\n".format(out_data2))
        self.out2.send(out_data2)

#PyProcModel implementing P3
@implements(proc=P3, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')       
class PyProcModelD(PyLoihiProcessModel):
    inp3: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    out3: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)
    def run_spk(self):
        in_data3 = self.inp3.recv()
        print("Received input data for P3: {}\n".format(in_data3))
        weight3 = np.array([2, 2])
        out_data3=in_data3-weight3
        print("Sent output data of P3: {}\n".format(out_data3))
        self.out3.send(out_data3)

#PyProcModel implementing P5
@implements(proc=P5, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')       
class PyProcModelE(PyLoihiProcessModel):
    inp5: PyInPort = LavaPyType(PyInPort.VEC_DENSE, float)
    out5: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, float)
    def run_spk(self):
        in_data5 = self.inp5.recv()
        print("P5 received: {}\n".format(in_data5))
        self.out5.send(in_data5)

# Containing process & model
A bigger process and its model is created, which encapsulates all the inside-steps (P2 & P3 in this case)

In [5]:
#Parent process.
class Combo(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get("shape", (2, ))
        self.s_in = InPort(shape=shape)
        self.s_out = OutPort(shape=shape)


In [6]:
#Parent process model.
import numpy as np
from lava.magma.core.model.sub.model import AbstractSubProcessModel

from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.decorator import implements

@implements(proc=Combo, protocol=LoihiProtocol)
class ComboModel(AbstractSubProcessModel):

    def __init__(self, proc):
 
        self.p2 = P2()
        self.p3 = P3()


        # connect Parent in port to child P2 in port
        proc.in_ports.s_in.connect(self.p2.inp2)
        # connect P2 Process to P3 Process
        self.p2.out2.connect(self.p3.inp3)
        # connect P3 Process to parent out port
        self.p3.out3.connect(proc.out_ports.s_out)

Next the processes are instantiated and the output _Port_ _out_ from _Process_ _P1_ is connected with the input _Port_ _inp_ of _the_ _hierarchical process_. At the end, the _output_ of _the_ _hierarchical process_ is connected to _Process_ _P5_ 

In [7]:
sender0 = P0()
sender1 = P1()
sender2 = Combo()
sender5 = P5()
# Connecting output port to an input port
sender0.out0.connect(sender2.s_in)
sender1.out1.connect(sender2.s_in)
sender2.s_out.connect(sender5.inp5)

Calling `run()` on either of these _Processes_ will first call the _Compiler_. During compilation the specified connection is setup by creating a channel between Processes. Now data can be transfered during execution as seen by the output print statements.

In [8]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

In [9]:
#sender2.run(RunSteps(num_steps=1), Loihi1SimCfg(select_tag='floating_pt', select_sub_proc_model=True))
sender1.run(RunSteps(num_steps=1), Loihi1SimCfg())
#sender1.stop()

Sent output data of P1: [1 2]
Sent output data of P0: [1.2 1. ]


P2 received: [2.2 3. ]

Sent output data of P2: [4.4 6. ]

Received input data for P3: [4.4 6. ]

Sent output data of P3: [2.4 4. ]

P5 received: [2.4 4. ]



The instance `sender0` of P0 sent the data `[1 1]` and the instance `sender1` of P1 sent the data `[1 2]`via their _OutPort_ `out0` & `out1` respectively to the _InPort_ `s_in` of the instance sender2 `recv` of the hierarchical process, where the data is received. After all the inside steps, the output data flows through the _OutPort_ `s_out` to the _InPort_ `inp5` of P5, instanciated by `sender5`